In [ ]:
import sys
sys.path.append('/home/jgould/Documents/AdaptiveLatents/tests')
from test_speed import get_speed_per_step, get_speed_by_time

import adaptive_latents as al
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import h5py


In [ ]:
A, beh, t, _ = al.input_sources.datasets.construct_unpublished24_data()

In [ ]:
# times_dict = get_speed_per_step(A, beh, bw_params=dict(num=1000), max_steps=2000)


In [ ]:
t = get_speed_by_time(A, beh, bw_params=dict(num=1000), max_steps=2000)
plt.plot((t[:-1] - t[0])*1000,np.diff(t)*1000);
plt.ylim([0, 70]);

In [ ]:
keys = list(times_dict.keys())
times = np.array([times_dict[k] for k in keys])*1000

In [ ]:
plt.hist(times.sum(axis=0), np.linspace(7,65,101));

In [ ]:
fig, ax = plt.subplots()
slice_start = 250
good_slice = times.sum(axis=0)[slice_start:]
ax.plot(np.arange(len(good_slice))+slice_start, good_slice)
ax.set_xlabel("iteration #");
ax.set_ylabel("iteration time (ms)")

In [ ]:
fig, ax = plt.subplots()
slice_start = 250
good_slice = times[:,slice_start:]
ax.plot(np.arange(good_slice.shape[1])+slice_start, good_slice.T, label=keys)
ax.set_xlabel("iteration #");
ax.set_ylabel("iteration time (ms)")
ax.set_ylim([0,100])
ax.legend();